# 目标规划
2020011054 金大为

（1）导入包和初始表

In [1]:
import pandas as pd
import numpy as np
data1=pd.read_excel('目标规划.xlsx')
cj=pd.read_excel('cj.xlsx')
sigma=pd.read_excel('检验数.xlsx')
R=4  #行数
C=10  #列数

cj表，第一行为P前面系数，第二行为优先级：

In [2]:
cj.set_index('C_j',inplace=True)
cj.fillna(0,inplace=True)
cj

,x1,x2,d1-,d1+,d2-,d2+,d3-,d3+,d4-,d4+
C_j,,,,,,,,,,
系数,0.0,0.0,0.0,1,1,0.0,1,1,1.5,1
P,0.0,0.0,0.0,2,3,0.0,4,1,4.0,1


原始单纯性表（C_B列代表P几）：

In [3]:
data1.fillna(0,inplace=True)
data1['C_B']=data1['C_B'].astype(np.int64)
data1

,系数,C_B,X_B,b,x1,x2,d1-,d1+,d2-,d2+,d3-,d3+,d4-,d4+,Q
0,0.0,0,d1-,40,1.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,3,d2-,100,1.0,1.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,4,d3-,30,1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0
3,1.5,4,d4-,15,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0


空的检验数表：

In [4]:
sigma.set_index('P',inplace=True)
sigma.fillna(0,inplace=True)
sigma

,x1,x2,d1-,d1+,d2-,d2+,d3-,d3+,d4-,d4+
P,,,,,,,,,,
P1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


d1将原始单纯性表中的系数表提取出来

In [5]:
d1=data1.loc[:,'x1':'d4+']
d1

,x1,x2,d1-,d1+,d2-,d2+,d3-,d3+,d4-,d4+
0,1.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0


（2）求检验数，第一次检验数表如下：

In [8]:
for i in range(R):#先处理系数表中数据
    P=int(data1.iloc[i,1])
    if P!=0:
        sigma.iloc[P-1]-=d1.iloc[i]*data1.iloc[i,0]
for i in range(C):#再处理cj中数据
    P=int(cj.iloc[1,i])
    if P!=0:
        sigma.iloc[P-1,i]+=cj.iloc[0,i]
sigma

,x1,x2,d1-,d1+,d2-,d2+,d3-,d3+,d4-,d4+
P,,,,,,,,,,
P1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
P2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
P3,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
P4,-1.0,-1.5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.5


（3）求入基向量：

In [9]:
r,c=0,0
flag=0#判断是否有检验数符合要求
for r in range(R):
    if flag:
        break
    for c in range(C):
        if sigma.iloc[r,c]<0:#遍历看是否有小于零的检验数
            if ((sigma.iloc[0:r,c]==0).all()):#前k-1行都为零
                flag=1
                break
                
c #入基向量所在列               

0

（4）求检验数，如下表Q：

In [10]:
data1.Q=data1.b/d1.iloc[:,c]
data1.replace(np.inf,np.nan,inplace=True)
data1

,系数,C_B,X_B,b,x1,x2,d1-,d1+,d2-,d2+,d3-,d3+,d4-,d4+,Q
0,0.0,0,d1-,40,1.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0
1,1.0,3,d2-,100,1.0,1.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,100.0
2,1.0,4,d3-,30,1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,30.0
3,1.5,4,d4-,15,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,NaN


（5）求出基向量：

In [11]:
for r in range(R):
    if data1.loc[r,'Q']<0:
        data1.loc[r,'Q']=np.nan#因为负数不参与比大小，将负数替换为NAN
r=data1.Q.argmin()
r#出基向量行

2

（6）入基并作初等行变换，最终表格如下：

In [12]:
data1.loc[r,'X_B']=cj.columns[c]
data1.loc[r,'C_B']=cj.iloc[1,c]
data1.loc[r,'系数']=cj.iloc[0,c]

In [13]:
data1.loc[r,'b':'d4+']/=d1.iloc[r,c]
for i in range(R):
    if i!=r:
        data1.loc[i,'b':'d4+']-=d1.iloc[i,c]*data1.loc[r,'b':'d4+']
    


In [14]:
data1

,系数,C_B,X_B,b,x1,x2,d1-,d1+,d2-,d2+,d3-,d3+,d4-,d4+,Q
0,0.0,0.0,d1-,10.0,0.0,1.0,1.0,-1.0,0.0,0.0,-1.0,1.0,0.0,0.0,40.0
1,1.0,3.0,d2-,70.0,0.0,1.0,0.0,0.0,1.0,-1.0,-1.0,1.0,0.0,0.0,100.0
2,0.0,0.0,x1,30.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,30.0
3,1.5,4.0,d4-,15.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,NaN


（7）用while循环嵌套重复上述步骤求出最优解：

In [15]:
flag=1
while flag:
    sigma.iloc[:,:]=0
    d1=data1.loc[:,'x1':'d4+']
    for i in range(R):
        P=int(data1.iloc[i,1])
        if P!=0:
            sigma.iloc[P-1]-=d1.iloc[i]*data1.iloc[i,0]
    for i in range(C):
        P=int(cj.iloc[1,i])
        if P!=0:
            sigma.iloc[P-1,i]+=cj.iloc[0,i]
    r,c=0,0
    flag=0#若没有符合要求的检验数，则flag一直为0，下一遍会退出循环，即为最优解
    for r in range(R):
        if flag:
            break
        for c in range(C):
            if sigma.iloc[r,c]<0:
                if ((sigma.iloc[0:r,c]==0).all()):
                    flag=1
                    break
    if flag:
        data1.Q=data1.b/d1.iloc[:,c]
        data1.replace(np.inf,np.nan,inplace=True)    
        for r in range(R):
            if data1.loc[r,'Q']<0:
                data1.loc[r,'Q']=np.nan
        r=data1.Q.argmin()
        
        data1.loc[r,'X_B']=cj.columns[c]
        data1.loc[r,'C_B']=cj.iloc[1,c]
        data1.loc[r,'系数']=cj.iloc[0,c]
        
        data1.loc[r,'b':'d4+']/=d1.iloc[r,c]
        for i in range(R):
            if i!=r:
                data1.loc[i,'b':'d4+']-=d1.iloc[i,c]*data1.loc[r,'b':'d4+']
    
        
data1

,系数,C_B,X_B,b,x1,x2,d1-,d1+,d2-,d2+,d3-,d3+,d4-,d4+,Q
0,0.0,0.0,x2,15.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,NaN
1,1.0,3.0,d2-,60.0,0.0,0.0,-1.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,NaN
2,0.0,0.0,x1,25.0,1.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,1.0,30.0
3,1.0,4.0,d3-,5.0,0.0,0.0,-1.0,1.0,0.0,0.0,1.0,-1.0,1.0,-1.0,5.0


（7）由上表可得出最优解为x1=25，x2=15，P3，P4等级目标不能实现。